In [1]:
import pandas as pd
import jinja2 as j2

df = pd.read_json("../all_modules.json")

In [2]:
# remove the mod_name and pkg_version columns
df = df.drop(columns=["mod_name", "pkg_version", "hash", "family"])
# keep only unique rows
df = df.drop_duplicates()

# filter out the highest version
df = df[df["version"] != df["version"].max()]

# sort by version descending, and  port and board
# df = df.sort_values(by=['version', 'port', 'board'], ascending=[False, True, True])

In [3]:
df.head()

,version,port,board,package
0,1.22.1,esp32,GENERIC,micropython-esp32-stubs
86,1.23.0,samd,seeed_wio_terminal,micropython-samd-seeed_wio_terminal-stubs
143,1.22.2,rp2,GENERIC,micropython-rp2-stubs
207,1.22.2,esp32,esp32_generic_c3,micropython-esp32-esp32_generic_c3-stubs
292,1.22.2,rp2,rpi_pico_w,micropython-rp2-rpi_pico_w-stubs


In [9]:



grouped = df.groupby('version')


# create a template environment
template_env = j2.Environment(loader=j2.FileSystemLoader('./templates'))
# load the template file
template = template_env.get_template('module_template.md')


# create a list to store the rendered modules
rendered_modules = []

# iterate over each group in the grouped dataframe
for version, group in grouped:
    # render the template with the group data
    rendered_module = template.render(version=version, modules=group.to_dict(orient='records'))
    # append the rendered module to the list
    rendered_modules.append(rendered_module)

# join the rendered modules into a single string
rendered_modules_str = '\n'.join(rendered_modules)


# #display rendered_modules_str as markdown output in jupyter notebook
from IPython.display import Markdown
Markdown(rendered_modules_str)

# save to docs/published_body.md
with open('published_body.md', 'w') as file:
    file.write(rendered_modules_str)
    
